<a href="https://colab.research.google.com/github/Sajag28/Web_Scrapped_Data_Sentimental_Analysis/blob/main/Web_Scrapped_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install openpyxl

In [ ]:
file_path ='/content/drive/MyDrive/Black_Trial/Input.xlsx'
import openpyxl
links=[]
workbook = openpyxl.load_workbook(file_path)
worksheet = workbook['Sheet1']
for i in range(2,101):
 cell_value = worksheet[f'B{i}'].value
 links.append(cell_value)
for i in range(0,5):
  print(links[i])


https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/
https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/
https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/
https://insights.blackcoffer.com/rise-of-cybercrime-and-its-effect-in-upcoming-future/
https://insights.blackcoffer.com/ott-platform-and-its-impact-on-the-entertainment-industry-in-future/


In [ ]:
!pip install beautifulsoup4

In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('cmudict')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

In [ ]:
!pip install nltk

In [ ]:
from bs4 import BeautifulSoup
from string import punctuation
import nltk
import re
import requests
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

def calculate_readability_metrics(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    avg_sentence_length = sum(len(s.split()) for s in sentences) / len(sentences)
    num_complex_words = sum(len(w) > 2 for w in words)
    fog_index = 0.4 * (avg_sentence_length + 100 * (num_complex_words / len(words)))
    avg_words_per_sentence = len(words) / len(sentences)
    # syllable_per_word = sum(nltk.corpus.cmudict.dict.get(w.lower(), ())[0][-1] for w in words if w.lower() in nltk.corpus.cmudict.dict) / len(words)
    personal_pronouns = sum(w.lower() in {'i', 'we', 'you', 'he', 'she', 'it', 'they', 'me', 'us', 'him', 'her', 'them'} for w in words)
    avg_word_length = sum(len(w) for w in words) / len(words)

    return {
        "avg_sentence_length": avg_sentence_length,
        "percentage_complex_words": num_complex_words / len(words) * 100,
        "fog_index": fog_index,
        "avg_words_per_sentence": avg_words_per_sentence,
        "complex_word_count": num_complex_words,
        "word_count": len(words),
        # "syllable_per_word": syllable_per_word,
        "personal_pronouns": personal_pronouns,
        "avg_word_length": avg_word_length,
    }

results=[]
for i in range(2,87):
    url = links[i]
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        title_text = soup.title.text.strip()
        div_with_class = soup.find('div', class_="td-post-content tagdiv-type")

        if div_with_class:
            div_paragraphs = div_with_class.find_all(['p','strong','li'])
            div_text = '\n\n'.join(p.get_text(strip=True) for p in div_paragraphs)
            clean_text = title_text + '\n' + div_text


            stop_words = []
            for filename in [
                "StopWords_Auditor.txt",
                "StopWords_Currencies.txt",
                "StopWords_DatesandNumbers.txt",
                "StopWords_GenericLong.txt",
                "StopWords_Generic.txt",
                "StopWords_Geographic.txt",
                "StopWords_Names.txt",
            ]:
                try:
                    with open("/content/drive/MyDrive/Black_Trial/" + filename, "r", encoding="utf-8") as file:
                        stop_words.extend(file.read().splitlines())
                except UnicodeDecodeError:
                    # print(f"Warning: Unable to decode {filename} with UTF-8 encoding.")
                    print(" ")

            pattern = re.compile('|'.join(re.escape(word) for word in stop_words))
            filtered_content = pattern.sub(' ', clean_text)


            # print(clean_text)
            words_to_include = set()
            for filename in ["negative-words.txt", "positive-words.txt"]:
              with open("/content/drive/MyDrive/Black_Trial/MasterDictionary/" + filename, "r", encoding="utf-8") as file:
                try:
                 with open("/content/drive/MyDrive/Black_Trial/MasterDictionary/" + filename, "r", encoding="utf-8") as file:
                    words_to_include.update(file.read().splitlines())
                except UnicodeDecodeError:
                  #  print(f"Warning: Unable to decode {filename} with UTF-8 encoding. Trying a different encoding.")
                   try:
                      with open("/content/drive/MyDrive/Black_Trial/MasterDictionary/" + filename, "r", encoding="latin-1") as file:
                        words_to_include.update(file.read().splitlines())
                   except UnicodeDecodeError:
                    #  print(f"Error: Unable to decode {filename} with either UTF-8 or latin-1 encoding.")
                    print(" ")
                for word in clean_text.split():
                  if word in words_to_include:
                   filtered_content=filtered_content+word


            final_filtered_content = ' '.join(filtered_content)

            # print(f"Filtered content (including specified words): {final_filtered_content}")
            from nltk.sentiment.vader import SentimentIntensityAnalyzer
            analyzer = SentimentIntensityAnalyzer()
            scores = analyzer.polarity_scores(final_filtered_content)
            readability_metrics = calculate_readability_metrics(final_filtered_content)

            results.append({
                "URL": url,
                "POSITIVESCORE":scores['pos'],
                "NEGATIVESCORE":scores['neg'],
                "POLARITYSCORE": scores['compound'],  # Add polarity score
                "SUBJECTIVITYSCORE": scores['neu'],
                "AVERAGE SENTENCE LENGTH": readability_metrics['avg_sentence_length'],
                "PERCENTAGE OF COMPLEX WORDS":readability_metrics['percentage_complex_words'],
                "FOGINDEX":readability_metrics['fog_index'],
                "AVG NUMBER OF WORDS":readability_metrics['avg_words_per_sentence'],
                "COMPLEX WORD COUNT":readability_metrics['complex_word_count'],
                "PERSONAL PRONOUNS":readability_metrics['personal_pronouns'],
                "AVG WORD LENGTH":readability_metrics['avg_word_length']
            })
            # print(scores)
            # print(results)
            df = pd.DataFrame(results)
            df.to_excel('sentiment_data_final_2.xlsx', index=False)

        else:
            print("Div with class 'td-post-content tagdiv-type' not found on the page.")
    else:
        print("Error: Unable to fetch the page.")



